**Cleaning EPL game data and feature engineering**

This code firstly cleans the team names used in the data to ensure they are consistent.

It then conducts some feature engineering, including:

1. Transforming variables into an appropriate numeric format to support Machine Learning prediction.

2. Merging the team and match data with the data we collected on points per game in the previous season as an indication of team strength.

3. Adding a feature to consider team tiredness based on the number of days since their previous game.

4. Creating features based on past performance and team form, which is hypothesised to be a particularly important predictor of match results.

In [1]:
# import packages
import pandas as pd 
import numpy as np 
import csv
from datetime import datetime
import time

# let's configure python to display to 1 decimpal places
pd.set_option("display.precision", 2)

# conigure pandas to display all columns so we can view the whole dataset
pd.set_option("display.max.columns", None)

# Configure python to display all rows up to 1000
pd.set_option("display.max.rows", 1000)

In [2]:
# Load raw basic match data from directory
data = pd.read_csv("../data/raw_EPL_data.csv", index_col = 0) # first column is an index column
# Load raw PL table data from directory
pl_tables = pd.read_csv("../data/raw_EPL_tables_data.csv", index_col = 0) # first column is an index column

FileNotFoundError: [Errno 2] No such file or directory: '2022-11-02_raw_EPL_data.csv'

***Cleaning team names***

One issue we have with the data in its raw format is that the team names are different in the 'team' and 'opponent' column, which makes it dificult to map across data in future. So let's create a mapping to ensure all our team names are consistent.

In [3]:
# collect team names from dataset containing basic game data
teams = np.sort(data.team.unique())
print(teams)

['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford'
 'Brighton and Hove Albion' 'Burnley' 'Cardiff City' 'Chelsea'
 'Crystal Palace' 'Everton' 'Fulham' 'Huddersfield Town' 'Leeds United'
 'Leicester City' 'Liverpool' 'Manchester City' 'Manchester United'
 'Newcastle United' 'Norwich City' 'Nottingham Forest' 'Sheffield United'
 'Southampton' 'Stoke City' 'Swansea City' 'Tottenham Hotspur' 'Watford'
 'West Bromwich Albion' 'West Ham United' 'Wolverhampton Wanderers']


In [4]:
# collect opponent names from dataset containing basic game data
pl_data = data[data['comp'] == "Premier League"] # filter to just contain PL opponents otherwise we get european and cup game opponents as well
opponents = np.sort(pl_data.opponent.unique())
print(opponents)

['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Brighton' 'Burnley'
 'Cardiff City' 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham'
 'Huddersfield' 'Leeds United' 'Leicester City' 'Liverpool'
 'Manchester City' 'Manchester Utd' 'Newcastle Utd' 'Norwich City'
 "Nott'ham Forest" 'Sheffield Utd' 'Southampton' 'Stoke City'
 'Swansea City' 'Tottenham' 'Watford' 'West Brom' 'West Ham' 'Wolves']


In [5]:
# the team and opponent names are not equal
np.array_equal(teams, opponents)

False

In [6]:
# Create a mapping to map old names onto consistent new names
class MissingDict(dict):
    __missing__ = lambda self, key: key

# set out all the inconsistent names we want to change
map_values = {"Brighton and Hove Albion": "Brighton",
              "Huddersfield Town": "Huddersfield",
              "Manchester United": "Manchester Utd", 
              "Newcastle United": "Newcastle Utd", 
              "Tottenham Hotspur": "Tottenham",
              "West Ham United": "West Ham",
              "Wolverhampton Wanderers": "Wolves",
              "Sheffield United": "Sheffield Utd",
              "West Bromwich Albion": "West Brom",
              "Nott'ham Forest": "Nottingham Forest"} 
mapping = MissingDict(**map_values)

# using this MissingDict mapping will ensure that teams that are not listed keep their original names
data["team"] = data["team"].map(mapping)
data["opponent"] = data["opponent"].map(mapping)

In [7]:
# collect new team names
teams = np.sort(data.team.unique())

# collect new opponent names
pl_data = data[data['comp'] == "Premier League"] # filter to just contain PL opponents otherwise we get european and cup game opponents as well
opponents = np.sort(pl_data.opponent.unique())

In [8]:
# the team and opponent names are now equal
np.array_equal(teams, opponents)

True

In [9]:
# Repeat the same mapping for the premier league standings data
# rename the column to aling with basic game data
pl_tables.rename(columns = {'squad': 'team',}, inplace = True)
# using this MissingDict mapping will ensure that teams that are not listed keep their original names
pl_tables["team"] = pl_tables["team"].map(mapping)

**Feature Engineering**

***Filtering out irrelevant features and transforming variables to support ML prediction***

As a first step, we want to remove any unecessary variables from our dataframe and then transform our remaining relevant columns into a numeric format to support our ML prediction models.

In [10]:
# First let's transform our string features to integer codes to support prediction
# Convert venue (home or away) to an integer code
data["venue_code"] = data["venue"].astype("category").cat.codes
# Calculate number of points taken from the game
conditions = [
    (data["result"] == "W"),
    (data["result"] == "L"),
    (data["result"] == "D")
]
values = [3, 0, 1]
data["points"] = np.select(conditions, values) 

In [11]:
# Now let's collect only relevant columns in an intuitive ordering
data = data[["season", "comp", "round", "date", "team", "venue", "venue_code", "opponent", "result", "points", "g", "g_a", "xg", "xg_a", "poss", "sh", "sot", "sh_a", "sot_a"]]
# and convert some columns that are meant to be integers into integers (some league cup results are recorded in a strange format so we record these as NA as we dont need them)
data['g'] = data['g'].apply(pd.to_numeric, errors='coerce')
data['g_a'] = data['g_a'].apply(pd.to_numeric, errors='coerce')

***Merge basic game data with PL points data from previous season as an indication of team strength***

One important aspect of prediction will be to tell how strong each team and opponent are. A proxy for this is their points per game across the previous season so here we merge the basic game data with data on points per game from the previous PL season to use as a feature in our model.

One issue is that we won't have data for newly promoted teams in each season. Instead, we will just use the average points per game of the 3 relegated sides as a proxy.

In [12]:
# add 1 to the season given we want to map to the points of the previous season (i.e., 2022 in the main data maps to the teams' final points in the 2021 season)
pl_tables['season'] = pl_tables['season'] + 1 

In [13]:
# merge with original data fram
data = data.merge(pl_tables[["season", "team", "pts/mp"]], how = 'left', left_on = ['season', 'team'], right_on = ['season', 'team'])

# rename columns
data.rename(columns = {'pts/mp': 'prev_season_ppg'}, inplace = True)

In [14]:
# repeat to merge on opponents
data = data.merge(pl_tables[["season", "team", "pts/mp"]], how = 'left', left_on = ['season', 'opponent'], right_on = ['season', 'team'])

# rename columns
data.rename(columns = {'pts/mp': 'prev_season_ppg_a',
                       'team_x': 'team',}, inplace = True)

del data['team_y']

In [15]:
# now we want to calculate the average points per game for the bottom 3 sides in each season
relegated_ppg = pl_tables[['season', 'pts/mp']].groupby('season').apply(lambda x: x.tail(3).mean())

# create a dictionary that maps the season onto the points per game that a previously relegated team should get for that season
relegated_ppg = relegated_ppg.iloc[:,1:] # drop first column
relegated_ppg = relegated_ppg.reset_index(level=0) # replace with index column of season
ppg_dict = relegated_ppg.set_index('season').to_dict()['pts/mp'] # transform into dictionary

# now map the NA values from each season onto the average points of the relegated sides from the previous season
data['prev_season_ppg'] = data['prev_season_ppg'].fillna(data['season'].map(ppg_dict))
data['prev_season_ppg_a'] = data['prev_season_ppg_a'].fillna(data['season'].map(ppg_dict))

***Include a feature to capture potential tiredeness from previous games***

Many teams have midweek fixtures for European or cup games, and broader fixture congestion can become a problem. A potential predictor of a result might therefore be how recently a team played. Here, we will include a feature based on the number of days since the previous game.

In [16]:
# Convert date object to a pandas datatime object
data["date"] = pd.to_datetime(data["date"])

# Calculate the number of days since previous game
data['days_since_last_game'] = (data.sort_values(['date']).
                                groupby(['team', 'season'])['date'].
                                apply(lambda x: x.diff()).dt.days)

# now remove data on all other comps as we dont need it
data = data[data['comp'] == "Premier League"]

C:\Users\lucel\AppData\Local\Temp\ipykernel_22608\122123878.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data['days_since_last_game'] = (data.sort_values(['date']).


***Use past game performance as a measure of pre-game form to feature in the model***

Here, we bring together data from the previous 3 games to measure form. We also calculate an exponential moving average from the last 5 games to get a more whollistic measure of form.

In [17]:
# Define a function to calculate exponential moving averages based on previous games leading up to each match
def exponential_moving_averages(group, cols, new_cols):
    group = group.sort_values("date") # sort by date to ensure we have the right ordering of games
    ema_stats = group[cols].shift(1).ewm(span = 10, min_periods = 5).mean() # take the relevant columns defined in the function and calculate exponential moving average with a span of 5 and a minimum of 3 periods otherwise NA is returned
    group[new_cols] = ema_stats # assign the new exponential moving average columns to new columns (names defined in the function)
    #group = group.dropna(subset = new_cols) # drop any NAs 
    return group # return the dataframe

In [18]:
# Define the columns to calculate EMAs
cols = ["points", "g", "g_a", "xg", "xg_a", "poss", "sh", "sot", "sh_a", "sot_a", "prev_season_ppg_a"]

# Define the name of the new columns as all the original columns with "_ema" on the end
new_cols = [f"{c}_ema" for c in cols]

# run function on data based on pre-defined columns
data = data.groupby(['team', 'season']).apply(lambda x: exponential_moving_averages(x, cols, new_cols))

C:\Users\lucel\AppData\Local\Temp\ipykernel_22608\1896003067.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.groupby(['team', 'season']).apply(lambda x: exponential_moving_averages(x, cols, new_cols))


In [19]:
# Define a function to collect the stats from the previous 3 games to use as features
def calculate_previous_statistics(group, cols, new_cols_1, new_cols_2, new_cols_3):
    # Sort by date to make sure games are correctly ordered
    group = group.sort_values('date')
    
    # Collect stats from previous 3 games
    previous_stats_1 = group[cols].shift(1)
    previous_stats_2 = group[cols].shift(2)
    previous_stats_3 = group[cols].shift(3)
    
    # Create new set of columns based on prevous 3 games
    group[new_cols_1] = previous_stats_1
    group[new_cols_2] = previous_stats_2
    group[new_cols_3] = previous_stats_3
    
    return group

In [24]:
# Define the columns to collect data from previous games from
cols = ["prev_season_ppg_a", "days_since_last_game", "points", "g", "g_a", "xg", "xg_a", "poss", "sh", "sot", "sh_a", "sot_a"]

# Define the name of the new columns as all the original columns with "_i" on the end to indicate which previous game
new_cols_1 = [f"{c}_1" for c in cols]
new_cols_2 = [f"{c}_2" for c in cols]
new_cols_3 = [f"{c}_3" for c in cols]

# run function on data based on pre-defined columns
data = data.groupby(['team', 'season']).apply(lambda x: calculate_previous_statistics(x, cols, new_cols_1, new_cols_2, new_cols_3))

C:\Users\lucel\AppData\Local\Temp\ipykernel_22608\1553839994.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.groupby(['team', 'season']).apply(lambda x: calculate_previous_statistics(x, cols, new_cols_1, new_cols_2, new_cols_3))


In [25]:
# Next, we want to collect opponent data and merge onto each game so we can make better predictions based on how the opponent is performing
# first, collect only relevant column names for features we want
feature_cols = ['date', 'opponent'] + (list(data[ list(data[ list(data.loc[:,'days_since_last_game':])])].columns.values))
opponent_data = data[feature_cols]
opponent = data.merge(opponent_data, left_on = ["date", "team"], right_on = ["date", "opponent"])

# delete repeated columns
del opponent['opponent_y']

# rename repeated columns
opponent.rename(columns = {'opponent_x': 'opponent'}, inplace = True)

# rename all columns with '_x' as columns of team data
opponent.columns = opponent.columns.str.replace("_x", "_team")

# rename columns with '_y' as columns with opponent data
opponent.columns = opponent.columns.str.replace("_y", "_opponent")

data = opponent

In [26]:
data = data.dropna(axis = 0)
data.to_csv(f"../data/clean_EPL_data.csv") # store as csv with time stamp

In [27]:
data

,season,comp,round,date,team,venue,venue_code,opponent,result,points,g,g_a,xg,xg_a,poss,sh,sot,sh_a,sot_a,prev_season_ppg,prev_season_ppg_a,days_since_last_game_team,points_ema_team,g_ema_team,g_a_ema_team,xg_ema_team,xg_a_ema_team,poss_ema_team,sh_ema_team,sot_ema_team,sh_a_ema_team,sot_a_ema_team,prev_season_ppg_a_ema_team,prev_season_ppg_a_1_team,days_since_last_game_1_team,points_1_team,g_1_team,g_a_1_team,xg_1_team,xg_a_1_team,poss_1_team,sh_1_team,sot_1_team,sh_a_1_team,sot_a_1_team,prev_season_ppg_a_2_team,days_since_last_game_2_team,points_2_team,g_2_team,g_a_2_team,xg_2_team,xg_a_2_team,poss_2_team,sh_2_team,sot_2_team,sh_a_2_team,sot_a_2_team,prev_season_ppg_a_3_team,days_since_last_game_3_team,points_3_team,g_3_team,g_a_3_team,xg_3_team,xg_a_3_team,poss_3_team,sh_3_team,sot_3_team,sh_a_3_team,sot_a_3_team,days_since_last_game_opponent,points_ema_opponent,g_ema_opponent,g_a_ema_opponent,xg_ema_opponent,xg_a_ema_opponent,poss_ema_opponent,sh_ema_opponent,sot_ema_opponent,sh_a_ema_opponent,sot_a_ema_opponent,prev_season_ppg_a_ema_opponent,prev_season_ppg_a_1_opponent,days_since_last_game_1_opponent,points_1_opponent,g_1_opponent,g_a_1_opponent,xg_1_opponent,xg_a_1_opponent,poss_1_opponent,sh_1_opponent,sot_1_opponent,sh_a_1_opponent,sot_a_1_opponent,prev_season_ppg_a_2_opponent,days_since_last_game_2_opponent,points_2_opponent,g_2_opponent,g_a_2_opponent,xg_2_opponent,xg_a_2_opponent,poss_2_opponent,sh_2_opponent,sot_2_opponent,sh_a_2_opponent,sot_a_2_opponent,prev_season_ppg_a_3_opponent,days_since_last_game_3_opponent,points_3_opponent,g_3_opponent,g_a_3_opponent,xg_3_opponent,xg_a_3_opponent,poss_3_opponent,sh_3_opponent,sot_3_opponent,sh_a_3_opponent,sot_a_3_opponent
5,2022,Premier League,Matchweek 6,2022-09-04,Arsenal,Away,0,Manchester Utd,L,0,1.0,3.0,1.3,1.5,60.0,16.0,3.0,10.0,6.0,1.82,1.53,4.0,3.00,2.51,0.84,2.10,0.59,58.09,18.45,6.69,7.11,2.33,1.02,1.18,4.0,3.0,2.0,1.0,2.4,0.4,59.0,22.0,8.0,4.0,3.0,0.70,7.0,3.0,2.0,1.0,2.6,0.8,71.0,22.0,8.0,11.0,3.0,0.70,7.0,3.0,3.0,0.0,1.3,0.3,57.0,14.0,6.0,6.0,1.0,3.0,2.14,1.03,1.08,1.36,1.22,49.59,12.21,3.75,14.10,4.09,1.47,1.37,5.0,3.0,1.0,0.0,1.5,0.7,47.0,9.0,2.0,10.0,2.0,1.05,5.0,3.0,1.0,0.0,1.1,1.4,51.0,11.0,4.0,17.0,4.0,2.42,9.0,3.0,2.0,1.0,1.8,1.3,30.0,13.0,5.0,17.0,5.0
6,2022,Premier League,Matchweek 8,2022-09-18,Arsenal,Away,0,Brentford,W,3,3.0,0.0,1.5,0.5,63.0,13.0,7.0,5.0,2.0,1.82,1.21,10.0,2.22,2.12,1.40,1.89,0.83,58.59,17.81,5.73,7.86,3.28,1.15,1.53,4.0,0.0,1.0,3.0,1.3,1.5,60.0,16.0,3.0,10.0,6.0,1.18,4.0,3.0,2.0,1.0,2.4,0.4,59.0,22.0,8.0,4.0,3.0,0.70,7.0,3.0,2.0,1.0,2.6,0.8,71.0,22.0,8.0,11.0,3.0,15.0,1.61,2.63,1.52,1.78,1.35,45.49,12.89,5.14,15.25,5.85,1.12,1.00,4.0,3.0,5.0,2.0,2.3,1.7,32.0,13.0,7.0,17.0,6.0,1.26,3.0,1.0,1.0,1.0,1.4,1.0,51.0,9.0,3.0,13.0,4.0,1.03,4.0,1.0,1.0,1.0,2.1,1.1,58.0,20.0,5.0,14.0,7.0
7,2022,Premier League,Matchweek 9,2022-10-01,Arsenal,Home,1,Tottenham,W,3,3.0,1.0,2.4,1.6,64.0,22.0,9.0,6.0,2.0,1.82,1.87,13.0,2.41,2.33,1.06,1.80,0.75,59.65,16.65,6.04,7.17,2.97,1.16,1.21,10.0,3.0,3.0,0.0,1.5,0.5,63.0,13.0,7.0,5.0,2.0,1.53,4.0,0.0,1.0,3.0,1.3,1.5,60.0,16.0,3.0,10.0,6.0,1.18,4.0,3.0,2.0,1.0,2.4,0.4,59.0,22.0,8.0,4.0,3.0,14.0,2.50,2.84,1.09,1.90,1.14,49.49,15.94,7.22,14.91,3.55,1.19,1.37,4.0,3.0,6.0,2.0,2.2,1.8,44.0,16.0,11.0,18.0,6.0,0.70,3.0,3.0,2.0,1.0,2.9,0.7,52.0,23.0,10.0,9.0,3.0,1.47,3.0,1.0,1.0,1.0,0.6,1.2,61.0,12.0,3.0,14.0,4.0
8,2022,Premier League,Matchweek 10,2022-10-09,Arsenal,Home,1,Liverpool,W,3,3.0,2.0,2.7,1.1,43.0,10.0,6.0,8.0,3.0,1.82,2.42,3.0,2.54,2.48,1.05,1.94,0.94,60.64,17.87,6.71,6.91,2.75,1.32,1.87,13.0,3.0,3.0,1.0,2.4,1.6,64.0,22.0,9.0,6.0,2.0,1.21,10.0,3.0,3.0,0.0,1.5,0.5,63.0,13.0,7.0,5.0,2.0,1.53,4.0,0.0,1.0,3.0,1.3,1.5,60.0,16.0,3.0,10.0,6.0,5.0,1.48,2.57,1.33,1.86,1.19,64.52,19.12,6.93,8.27,3.57,1.15,1.34,18.0,1.0,3.0,3.0,1.5,1.4,54.0,15.0,7.0,6.0,6.0,1.03,3.0,1.0,0.0,0.0,2.1,1.7,61.0,23.0,8.0,14.0,3.0,1.29,4.0,3.0,2.0,1.0,1.5,0.7,72.0,23.0,6.0,5.0,2.0
9,2022,Premier League,Matchweek 11,2022-10-16,Arsenal,Away,0